# Exploration des Données pour l'Optimisation de Portefeuille

Ce notebook explore les données de marché et prépare les inputs nécessaires pour les différentes méthodes d'optimisation de portefeuille.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

# Ajouter le chemin du projet au PYTHONPATH
sys.path.append(os.path.abspath('../'))

# Configuration des plots
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Importer les modules personnalisés
from src.data.data_collection import MarketDataCollector
from src.data.data_processing import DataProcessor

## 1. Collecte des Données

Nous commençons par collecter les données historiques pour un ensemble d'actions et d'obligations.

In [ ]:
# Définir la liste des actifs
# Exemples d'actions américaines (S&P 500)
stocks = [
    'AAPL', 'MSFT', 'AMZN', 'GOOGL', 'META', 'TSLA', 'NVDA', 'BRK-B', 'JPM', 'JNJ', 
    'PG', 'V', 'UNH', 'HD', 'BAC', 'XOM', 'PFE', 'DIS', 'CSCO', 'VZ'
]

# ETFs d'obligations (pour représenter différentes classes d'obligations)
bonds = [
    'AGG',  # iShares Core U.S. Aggregate Bond ETF
    'BND',  # Vanguard Total Bond Market ETF
    'GOVT', # iShares U.S. Treasury Bond ETF
    'TIP',  # iShares TIPS Bond ETF
    'HYG',  # iShares iBoxx $ High Yield Corporate Bond ETF
    'LQD',  # iShares iBoxx $ Investment Grade Corporate Bond ETF
    'MUB',  # iShares National Muni Bond ETF
    'BWX',  # SPDR Bloomberg International Treasury Bond ETF
    'EMB',  # iShares J.P. Morgan USD Emerging Markets Bond ETF
    'VCSH'  # Vanguard Short-Term Corporate Bond ETF
]

# Autres actifs (Or, Immobilier, Matières premières)
alternatives = [
    'GLD',  # SPDR Gold Shares
    'VNQ',  # Vanguard Real Estate ETF
    'GSG',  # iShares S&P GSCI Commodity-Indexed Trust
    'BTC-USD' # Bitcoin
]

all_assets = stocks + bonds + alternatives

# Période de collecte des données
start_date = '2015-01-01'
end_date = '2025-01-01'

In [ ]:
# Initialiser le collecteur de données
data_collector = MarketDataCollector()

# Collecter les prix historiques
prices_df = data_collector.download_data(all_assets, start_date, end_date, interval='1d')

# Afficher les premières lignes
prices_df.head()

In [ ]:
# Sauvegarder les données brutes
os.makedirs('../data/raw', exist_ok=True)
prices_df.to_csv('../data/raw/historical_prices.csv')

## 2. Préparation et Nettoyage des Données

In [ ]:
# Initialiser le processeur de données
data_processor = DataProcessor(prices_df)

# Calculer les rendements
returns_df = data_processor.calculate_returns(method='log')

# Afficher les premières lignes
returns_df.head()

In [ ]:
# Vérifier les données manquantes
missing_data = returns_df.isnull().sum()
print("Nombre de valeurs manquantes par actif:")
print(missing_data[missing_data > 0])

In [ ]:
# Traiter les valeurs manquantes
returns_df_clean = data_processor.handle_missing_values(returns_df, method='ffill')

# Vérifier qu'il n'y a plus de valeurs manquantes
print("Nombre de valeurs manquantes après traitement:")
print(returns_df_clean.isnull().sum().sum())

## 3. Analyse Exploratoire

In [ ]:
# Statistiques descriptives des rendements
returns_stats = returns_df_clean.describe().T
returns_stats['annualized_return'] = returns_df_clean.mean() * 252
returns_stats['annualized_volatility'] = returns_df_clean.std() * np.sqrt(252)
returns_stats['sharpe_ratio'] = returns_stats['annualized_return'] / returns_stats['annualized_volatility']

# Trier par ratio de Sharpe
returns_stats.sort_values('sharpe_ratio', ascending=False).head(10)

In [ ]:
# Visualiser la distribution des rendements
plt.figure(figsize=(15, 10))

# Sélectionner un sous-ensemble d'actifs pour la lisibilité
selected_assets = ['AAPL', 'MSFT', 'AMZN', 'AGG', 'BND', 'GLD', 'VNQ']
selected_returns = returns_df_clean[selected_assets]

# Tracé des distributions
for column in selected_returns.columns:
    sns.kdeplot(selected_returns[column], label=column)

plt.title('Distribution des Rendements Journaliers')
plt.xlabel('Rendement')
plt.ylabel('Densité')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Matrice de corrélation
corr_matrix = returns_df_clean.corr()

# Visualisation de la matrice de corrélation
plt.figure(figsize=(16, 14))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
cmap = sns.diverging_palette(230, 20, as_cmap=True)

sns.heatmap(corr_matrix, mask=mask, cmap=cmap, vmax=1, vmin=-1, center=0,
            square=True, linewidths=.5, annot=False, fmt='.2f')

plt.title('Matrice de Corrélation des Rendements')
plt.tight_layout()
plt.show()

## 4. Préparation des Données pour l'Optimisation

In [ ]:
# Calculer les rendements moyens annualisés
expected_returns = returns_df_clean.mean() * 252

# Calculer la matrice de covariance annualisée
cov_matrix = returns_df_clean.cov() * 252

# Afficher les rendements attendus
expected_returns.sort_values(ascending=False).head(10)

In [ ]:
# Sauvegarder les données traitées
os.makedirs('../data/processed', exist_ok=True)
returns_df_clean.to_csv('../data/processed/cleaned_returns.csv')

# Sauvegarder les rendements attendus et la matrice de covariance
expected_returns.to_csv('../data/processed/expected_returns.csv')
cov_matrix.to_csv('../data/processed/covariance_matrix.csv')

## 5. Analyse des classes d'actifs

In [ ]:
# Créer des dataframes par classe d'actifs
stocks_returns = returns_df_clean[stocks]
bonds_returns = returns_df_clean[bonds]
alts_returns = returns_df_clean[alternatives]

# Calculer les rendements et volatilités moyens par classe d'actifs
asset_classes = {
    'Actions': stocks_returns,
    'Obligations': bonds_returns,
    'Alternatifs': alts_returns
}

# Calculer les statistiques par classe d'actifs
class_stats = {}
for name, df in asset_classes.items():
    ret = df.mean() * 252
    vol = df.std() * np.sqrt(252)
    sharpe = ret / vol
    class_stats[name] = {
        'return': ret.mean(),
        'volatility': vol.mean(),
        'sharpe': sharpe.mean()
    }

# Convertir en DataFrame pour l'affichage
class_stats_df = pd.DataFrame(class_stats).T
class_stats_df

In [ ]:
# Visualiser les rendements cumulés par classe d'actifs
plt.figure(figsize=(14, 8))

# Calculer les rendements cumulés (stratégie équipondérée au sein de chaque classe)
cumulative_returns = {}
for name, df in asset_classes.items():
    # Moyenne équipondérée des rendements journaliers
    avg_returns = df.mean(axis=1)
    # Rendement cumulé
    cumulative = (1 + avg_returns).cumprod() - 1
    plt.plot(cumulative.index, cumulative, label=name)

plt.title('Rendements Cumulés par Classe d\'Actifs')
plt.xlabel('Date')
plt.ylabel('Rendement Cumulé')
plt.legend()
plt.grid(True)
plt.show()

## 6. Conclusion

Ce notebook a permis d'explorer et de préparer les données pour les différentes méthodes d'optimisation de portefeuille. Nous avons:

1. Collecté les données historiques pour un ensemble diversifié d'actifs
2. Nettoyé et préparé les données en traitant les valeurs manquantes
3. Analysé les caractéristiques statistiques des rendements
4. Étudié les corrélations entre les actifs
5. Calculé les rendements attendus et la matrice de covariance pour l'optimisation
6. Comparé les performances des différentes classes d'actifs

Les prochains notebooks utiliseront ces données préparées pour implémenter et évaluer différentes méthodes d'optimisation de portefeuille.